## Random assignment, binary CATE example

This is a fully worked-out notebook showing how you would apply causaltune to a dataset.

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# the below checks for whether we run dowhy, causaltune, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "auto-causality"))

try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

try:
    import flaml
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "FLAML"))


In [ ]:
# this makes the notebook expand to full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%javascript

// turn off scrollable windows for large output
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [ ]:
from causaltune import CausalTune
from causaltune.datasets import synth_ihdp

### Model fitting & scoring
Here we fit a (selection of) model(s) to the data and score them with the energy distance metric on held-out data.

We import an example dataset and pre-process it. The pre-processing fills in the NaNs and one-hot-encodes all categorical and int variables.

If you don't want an int variable to be one-hot-encoded, please cast it to float before preprocessing.

In [ ]:
# load toy dataset and apply standard pre-processing
cd = synth_ihdp()
cd.preprocess_dataset()

Inspecting the dataset below, we can see that the causal inference problem is defined by a binary `treatment`, a continuous outcome `y_factual`, and a range of covariates `x_{i}`. The `random` column is added to bypass a DoWhy bug.

In [ ]:
# inspect the preprocessed dataset
display(cd.data.head())

Fitting the model is as simple as calling CausalTune.fit(), with the only required parameter apart from the data being the amount of time you want to give the optimizer, either for the whole run (`time_budget`) or per FLAML component model (`components_time_budget`), or both.

If you want to use specific estimators, comment in the `estimator_list` below to include any estimators whose full name contains any of the elements of `estimator_list`.

The other allowed values are `all` and `auto`, the default is `auto`.


In [ ]:
# training configs

# choose estimators of interest
estimator_list = [
            "Dummy",
            # "SparseLinearDML",
            # "ForestDRLearner",
            # "TransformedOutcome",
            # "CausalForestDML",
            # ".LinearDML",
            # "DomainAdaptationLearner",
            "SLearner",
            "XLearner",
            # "TLearner",
            # "Ortho"
    ]

# set evaluation metric
metric = "energy_distance"

# it's best to specify either time_budget or components_time_budget, 
# and let the other one be inferred; time in seconds
time_budget = None
components_time_budget = 10

# specify training set size
train_size = 0.7


In [ ]:
ct = CausalTune(
    estimator_list=estimator_list,
    metric=metric,
    verbose=3,
    components_verbose=2,
    time_budget=time_budget,
    components_time_budget=components_time_budget,
    train_size=train_size
)


# run causaltune
ct.fit(data=cd, outcome=cd.outcomes[0])

print('---------------------')
# return best estimator
print(f"Best estimator: {ct.best_estimator}")
# config of best estimator:
print(f"best config: {ct.best_config}")
# best score:
print(f"best score: {ct.best_score}")

After running a fit, you also have the option resume it without losing past results, for example if you want to search over extra estimators. To do so, simply pass `resume=True` to the `fit` method as shown below: 

`ct.fit(data=cd,outcome=cd.outcomes[0],resume=True)`

We will now score all estimators on the held-out test set and save the results per estimator in `ct.scores`.

In [ ]:
test_df = ct.test_df

for est_name, scr in ct.scores.items():
    causal_estimate = scr['estimator']
    scr['scores']['test'] = ct.scorer.make_scores(causal_estimate, test_df, metrics_to_report=ct.metrics_to_report)

Below we demonstrate some basic plotting functionality that comes with `causaltune`. 

In [ ]:
from causaltune.visualizer import Visualizer

viz = Visualizer(
    test_df=test_df,
    treatment_col_name=cd.treatment,
    outcome_col_name=cd.outcomes[0]
)

In [ ]:
%matplotlib inline

# plotting metrics by estimator

figtitle = f'{viz.outcome_col_name}'
figsize = (7,5)
metrics = ('energy_distance', 'ate')

viz.plot_metrics_by_estimator(
    scores_dict=ct.scores,
    metrics=metrics,
    figtitle=figtitle,
    figsize=figsize
)

In [ ]:
%matplotlib inline

# plotting shapley feature importances
# sampling from test_df as calculation can take a while
# optional: supply pre-computed shapley values by passing them as shaps=your_array

use_df = test_df.sample(100)
est = ct.model

viz.plot_shap(
    estimate=est,
    df=use_df,
    figtitle='Shapley feature importances'
)

In [ ]:
%matplotlib inline

# plotting out-of sample difference of outcomes between treated and untreated 
# for the points where model predicts positive vs negative impact

viz.plot_group_ate(
    scorer=ct.scorer,
    scores_dict=ct.scores,
    estimator=ct.best_estimator
)